In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from IPython.display import Image, display

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParms['axes.grid'] = False

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

class Forecasting:
    def __init__(self, df:pd.DataFrame, target):
        self.df = df
        self.target = target
        self.tts = dict()
        self.window_size = 5
        
    def data_to_X_y(self, data, window_size=5):
        data_as_np = data.to_numpy()
        X = []
        y = []
        for i in range(len(data_as_np)-window_size):
            row = [[a] for a in data_as_np[i:i+5]]
            X.append(row)
            label = data_as_np[i+5]
            y.append(label)
        return np.array(X), np.array(y)
    
    def split_train_test(self, X, y, train, test, validation):
        self.tts['xtrain'], self.tts['ytrain'] = X[:train], y[:train]
        self.tts['xtest'], self.tts['ytest'] = X[:test], y[:test]
        self.tts['xval'], self.tts['yval'] = X[:validation], y[:validation]
        
    def prepareLSTMForecasting(self, date_column, window_size=5, train_split, test_split, validation_split):
        # transform date column to index
        self.df.index = pd.to_datetime(self.df[date_column], format='%Y-%m-%d %H:%M:%S')
        
        # create time series
        if window_size:
            self.window_size = window_size
        X, y = self.data_to_X_y(self.df[self.target], window_size=self.window_size)
        print(X.shape, y.shape)
        
        # splitting data
        self.split_train_test(X, y, train_split, test_split, validation_split)

    def plotForecasting(self, x, y):
        plt.plot(x)
        plt.plot(y)
        plt.show()
        
    def saveModel(self, model, title):
        path_pickle = 'saved models/'+title+'.pkl'
        path_joblib = 'saved models/'+title+'.sav'
        pickle.dump(model, open(path_pickle, 'wb'))
        joblib.dump(model, open(path_joblib, 'wb'))
        print('model saved at',path_pickle,'and',path_joblib)
    
    def LSTMforecasting(self, epochs=5, plot=False):
        model = Sequential()
        model.add(InputLayer((self.window_size,1)))
        model.add(LSTM(64))
        model.add(Dense(8, 'relu'))
        model.add(Dense(1, 'linear'))
        print(model.summary())
        
        # callbacks for checking validity
        cb = ModelCheckpoint('model/', save_best_only=True)
        model.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
        
        # fitting the model
        model.fit(self.tts['xtrain'], self.tts['ytrain'], validation_data=[self.tts['xval'], self.tts['yval']], epochs=epochs, callbacks=[cb])
        
        model = load_model('model/')
        
        if plot:
            train_predictions = model.predict(self.tts['xtrain']).flatten()
            train_results = pd.DataFrame(data={'Train predictions':train_predictions, 'Actuals':self.tts['ytrain']})
            self.plotForecasting(train_results['Train predictions'], train_results['Actuals'])
            
            test_predictions = model.predict(self.tts['xtest']).flatten()
            test_results = pd.DataFrame(data={'Test predictions':train_predictions, 'Actuals':self.tts['ytest']})
            self.plotForecasting(train_results['Test predictions'], train_results['Actuals'])
            
            val_predictions = model.predict(self.tts['xval']).flatten()
            val_results = pd.DataFrame(data={'Val predictions':train_predictions, 'Actuals':self.tts['yval']})
            self.plotForecasting(train_results['Val predictions'], train_results['Actuals'])
        
        self.saveModel(model, 'lstmforecasting')
        return model        

In [ ]:
import seaborn as sns
class prophetForecasting:
    def __init__(self, df:pd.DataFrame, label):
        self.df = df
        self.y = None
        self.X = None
        self.label = label
        self.data = None
        self.tts = dict()
        
    def createFeatures(self, column):
        self.df.index = pd.to_datetime(self.df[column], format="%Y-%m-%d %H:%M:%S")
        df = self.df.copy()
        df['date'] = df.index
        df['hour'] = df['date'].dt.hour
        df['dayofweek'] = df['date'].dt.dayofweek
        df['quarter'] = df['date'].dt.quarter
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
        df['dayofyear'] = df['date'].dt.dayofyear
        df['dayofmonth'] = df['date'].dt.day
        df['weekofyear'] = df['date'].dt.weekofyear

        self.X = df[['hour','dayofweek','quarter','month','year',
               'dayofyear','dayofmonth','weekofyear']]
        self.y = df[self.label]
        self.data = pd.concat([self.X, self.y], axis=1)
        return self.data
    
    def plotTrends(self):
        sns.pairplot(self.data.dropna(),
             hue='hour',
             x_vars=['hour','dayofweek',
                     'year','weekofyear'],
             y_vars=self.label,
             height=5,
             plot_kws={'alpha':0.15, 'linewidth':0}
            )
        plt.suptitle('Journey Ticket Price, Day of Week, Year and Week of Year')
        plt.show()
        
    def split_train_test(self, split_date):
        self.tts['train'] = self.df.loc[self.df.index <= split_date].copy()
        self.tts['test'] = self.df.loc[self.df.index > split_date].copy()
        self.tts['test'] \
            .rename(columns={self.label: 'TEST SET'}) \
            .join(self.tts['train'].rename(columns={self.label+'_': 'TRAINING SET'}),
                  how='outer',lsuffix='_left', rsuffix='_right') \
            .plot(figsize=(15,5), title=self.label, style='.')
        plt.show()
        
    def prophetForecast(self):
        self.tts['train'].reset_index() \
            .rename(columns={'Datetime':'ds', self.label:'y'})
        model = Prophet()
        model.fit(self.tts['train'].reset_index() \
                      .rename(columns={'Datetime':'ds',
                                       self.label:'y'}))
        
        test_predict = model.predict(df=self.tts['test'].reset_index() \
                                       .rename(columns={'Datetime':'ds'}))
        f, ax = plt.subplots(1)
        f.set_figheight(5)
        f.set_figwidth(15)
        fig = model.plot(test_predict,ax=ax)
        plt.show()
        return model

obj = prophetForecasting(temp,'price')
obj.createFeatures('departureDate')
obj.plotTrends()
obj.split_train_test('2018-05-17 14:00:00')
obj.prophetForecast()